# Part 3: LGBM Model Development

In [51]:
import pandas as pd
ml_df = pd.read_csv('data/racedata_2017-2022_ETL.csv', index_col = 0)

In [52]:
ml_df = ml_df.dropna()

In [53]:
import os
import pickle
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error

from sklearn.feature_selection import RFE

working_directory = 'D:/machine_learning/DFS/nascar/'
data_dir = 'data/'

# needs hyperparameters

dcols = [
    'Pos',
     'St',
     '#',
     'Driver',
     'Laps',
     'Led',
     'Pts',
     'PPts',
     'Track',
     'race_ID',
     'FP',
     'place_diff',
     'Date',
     'Date_dtobj',
     'Year',
     'DNF',
     'Win',
     'T5',
     'T10',
     'T20',
     'track_type',
     'Start',
     'Mid Race',
     'Finish',
     'High Pos.',
     'Low Pos.',
     'Avg. Pos.',
     'Pass Diff.',
     'Green Flag Passes',
     'Green Flag Times Passed',
     'Quality Passes',
     'Pct. Quality Passes',
     'Fastest Lap',
     'Top 15 Laps',
     'Pct. Top 15 Laps',
     'Laps Led',
     'Pct. Laps Led',
     'Total Laps',
     'DRIVER RATING',
]

X = ml_df.drop(dcols, axis = 1)
Y = ml_df['FP']

from sklearn.model_selection import train_test_split

#Create Training and Testing DataSets
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.25, random_state=42)

X_train.reset_index(inplace = True, drop=True)
X_test.reset_index(inplace = True, drop=True)
Y_train.reset_index(inplace = True, drop=True)
Y_test.reset_index(inplace = True, drop=True)

print('Training set size:', len(X_train))
print('Testing set size:', len(X_test))

pred_df = pd.concat([X_test, Y_test], axis = 1)

print('\nNum Possible Features:',len(X_train.columns.tolist()))

Training set size: 6234
Testing set size: 2078

Num Possible Features: 103


In [54]:
# get statuses for comparison
train_status = X_train['Status']
test_status = X_test['Status']

X_train.drop(['Status'], axis = 1, inplace = True)
X_test.drop(['Status'], axis = 1, inplace = True)

pred_df['Status'] = test_status

C:\Users\Greg\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [55]:
# standardize data
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer
from sklearn.compose import ColumnTransformer

# get numerical features from the features dataframe and store numerical columns in a variable
numerical_features = X_train.select_dtypes(include=['float64', 'int64'])
numerical_columns = numerical_features.columns
 
ct = ColumnTransformer([("only numeric", StandardScaler(), numerical_columns)], remainder='passthrough')

# fit columntransformer to the training data and transform it
X_train_scaled = ct.fit_transform(X_train)

# fit columntransformer to the testing data and transform it
X_test_scaled = ct.transform(X_test)

# convert to pd dataframes
X_train_scaled = pd.DataFrame(X_train_scaled, columns = X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns = X_test.columns)

In [56]:
X_train_scaled

,LL_yr_cumsum,LC_yr_cumsum,DNF_yr_cumsum,Win_yr_cumsum,T5_yr_cumsum,T10_yr_cumsum,T20_yr_cumsum,FinPos_mean_L3,StrtPos_mean_L3,PlaceDiff_mean_L3,...,GFTP_sum_L3_tt,GFTP_mean_L3_tt,QP_sum_L3_tt,QP_mean_L3_tt,Pct_QP_mean_L3_tt,T15Laps_sum_L3_tt,T15Laps_mean_L3_tt,Pct_T15Laps_mean_L3_tt,DRating_sum_L3_tt,DRating_mean_L3_tt
0,-0.488737,-0.305571,-0.687274,-0.421026,-0.655717,-0.485303,-0.406760,0.443735,0.094085,-0.448399,...,0.104414,0.051271,-0.456574,-0.472166,-0.559088,-0.642191,-0.659511,-0.708645,-0.316494,-0.416126
1,-0.477410,0.270799,0.310506,-0.421026,-0.655717,-0.306698,0.201302,0.747097,-0.307039,-1.457496,...,0.204210,0.149527,-0.192576,-0.213698,-0.013247,-0.449686,-0.469587,-0.076067,0.031346,-0.052859
2,-0.462308,0.636707,0.310506,-0.421026,-0.655717,-0.663907,0.201302,-0.087148,0.428356,0.762516,...,0.126591,0.073106,-0.702971,-0.713403,-1.018680,-0.878165,-0.892321,-1.062174,-0.192890,-0.287040
3,-0.492512,-1.500034,0.310506,-0.421026,-0.655717,-0.842511,-1.136435,1.846785,1.531449,-0.145671,...,-0.361305,-0.407260,-0.755771,-0.765097,-1.155448,-0.899900,-0.913764,-1.013514,-1.058116,-1.190638
4,-0.484961,-0.644894,-1.186164,-0.421026,-0.379481,-0.485303,-0.649985,-0.580112,0.394929,1.367974,...,-0.865834,-0.292627,-0.447774,0.174004,0.315737,-0.195082,1.245855,0.276469,-1.178438,0.582287
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6229,-0.492512,-1.384833,-0.687274,-0.421026,-0.655717,-0.663907,-0.893210,0.822938,0.929762,0.308423,...,0.536867,0.477049,-0.007778,-0.032771,0.087789,-0.154718,-0.178575,-0.317380,0.117759,0.037387
6230,-0.488737,-0.980895,-0.687274,-0.421026,-0.655717,-0.842511,-0.771598,0.102453,0.628918,0.812971,...,1.052484,0.984708,0.555417,0.518627,-0.085943,-0.465210,-0.484904,-0.412714,-0.048504,-0.136250
6231,-0.488737,-1.538434,-1.186164,-0.421026,-0.655717,-0.842511,-1.014823,1.770944,1.498021,-0.095216,...,-0.355761,-0.401801,-0.738171,-0.747865,-1.076591,-0.918529,-0.932144,-1.118778,-1.018738,-1.149513
6232,-0.303736,0.356830,1.807175,-0.421026,-0.103245,0.586323,0.687753,-0.352590,-1.142716,-1.255676,...,-0.145079,-0.194370,0.089021,0.062001,1.191794,1.388430,1.343880,1.297334,0.972046,0.929561


In [57]:
X_test_scaled

,LL_yr_cumsum,LC_yr_cumsum,DNF_yr_cumsum,Win_yr_cumsum,T5_yr_cumsum,T10_yr_cumsum,T20_yr_cumsum,FinPos_mean_L3,StrtPos_mean_L3,PlaceDiff_mean_L3,...,GFTP_sum_L3_tt,GFTP_mean_L3_tt,QP_sum_L3_tt,QP_mean_L3_tt,Pct_QP_mean_L3_tt,T15Laps_sum_L3_tt,T15Laps_mean_L3_tt,Pct_T15Laps_mean_L3_tt,DRating_sum_L3_tt,DRating_mean_L3_tt
0,0.194634,0.439907,-0.687274,-0.421026,-0.103245,0.229114,1.174203,-0.276750,-1.376705,-1.709770,...,-0.117357,-0.167077,-0.254176,-0.274008,-0.052675,-0.266495,-0.288853,1.047083,0.790469,0.739931
1,2.550564,1.947847,-1.186164,1.339911,2.382877,2.550969,2.390328,-0.845554,-0.307039,0.661606,...,-0.211610,-0.259875,0.018622,-0.006924,-0.129069,0.422798,0.391197,0.454227,0.615455,0.557155
2,-0.492512,-0.650433,-0.188384,-0.421026,-0.379481,-0.663907,-0.893210,0.595416,-0.073050,-0.902493,...,-0.649607,-0.691112,-0.764571,-0.773712,-1.152984,-0.924739,-0.938271,-1.139633,-0.600892,-0.713136
3,-0.443430,-0.656710,-0.188384,-0.421026,-0.655717,0.050510,-0.285148,-0.049228,-1.276424,-1.861134,...,-0.621886,-0.390883,-0.051778,0.277391,2.481854,0.810914,1.636424,0.529699,-0.420409,0.345821
4,-0.488737,-1.202802,-1.186164,-0.421026,-0.655717,-0.485303,-0.649985,-0.921394,0.628918,2.175250,...,-0.849201,-0.887625,-0.702971,-0.713403,-0.936126,-0.750863,-0.766726,-0.956910,-0.385406,-0.488094
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2073,-0.484961,-0.658925,-1.186164,-0.421026,-0.655717,-0.842511,-0.528373,0.064533,-0.173331,-0.347490,...,-0.488823,-0.532810,-0.694172,-0.704787,-0.886840,-0.819172,-0.834119,-1.062174,-0.413846,-0.517795
2074,1.210251,0.057014,-0.188384,0.459443,1.277934,0.764927,0.201302,-0.693873,-0.607883,0.005694,...,-0.117357,-0.167077,0.687416,0.647861,1.572528,1.431899,1.386766,1.660793,1.746483,1.738344
2075,-0.488737,-0.850187,-0.687274,-0.421026,-0.655717,-0.842511,-1.014823,0.178293,0.929762,1.166154,...,-0.533177,-0.259875,-0.711771,-0.691864,-0.957688,-0.924739,-0.932144,-1.125233,-0.980453,-0.531503
2076,1.504743,1.893940,2.804954,0.459443,1.554170,2.193761,2.147103,-1.262677,-0.875299,0.358877,...,0.121046,0.067647,0.766615,0.725402,1.100615,1.916267,1.864639,1.538647,1.543029,1.525867


In [58]:
# # hyperparameter tuning function
# from sklearn.model_selection import GridSearchCV
# from sklearn.metrics import mean_absolute_error

# def hyperparameter_tuning(feat_set):
#     # define model for param tuning
#     model = LGBMRegressor()

#     # define param grid
#     param_grid = {
#         'random_state' : [1],
#         'n_jobs' : [-1],
#         'n_estimators' : [1000],
#         'learning_rate': [0.001, 0.01, 0.1],
#         'num_leaves': [15, 31, 62, 93, 127],
#         'max_depth' :[-1, 20, 40],
#         'min_child_samples' : [20, 80],
#         'max_bin' : [63],
#         'device' : ['gpu']
#     }

#     gsearch = GridSearchCV(model, param_grid, verbose = 10, scoring = 'neg_mean_absolute_error') 

#     lgb_model = gsearch.fit(X_train[feat_set], Y_train)
#     print('best params')
#     print(gsearch.best_params_, gsearch.best_score_)
#     preds_lgb_model = lgb_model.predict(X_test[feat_set])
#     mae_lgb = mean_absolute_error(Y_test, preds_lgb_model)
#     print(" MAE: %f" % (mae_lgb ))
#     return lgb_model.best_params_

In [59]:
import os
import joblib
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from lightgbm import LGBMRegressor
import xgboost as xgb
from sklearn.metrics import mean_absolute_error

from sklearn.feature_selection import RFE

working_directory = 'D:/machine_learning/DFS/nascar/ML'
os.chdir(working_directory)

import os
import joblib
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from lightgbm import LGBMRegressor
import xgboost as xgb
from sklearn.metrics import mean_absolute_error

from sklearn.feature_selection import RFE

working_directory = 'D:/machine_learning/DFS/nascar/ML'
os.chdir(working_directory)

# needs hyperparameters
def lgbm_mod(): return LGBMRegressor(random_state = 1,
                                     n_estimators = 1000,
                                     learning_rate = 0.1,
                                     n_jobs = -1,
                                     max_bin = 63,
                                     max_depth = 20,
                                     min_child_samples = 20,
                                     num_leaves = 127,
                                     device = 'gpu')

""" MODEL SELECTION """

model = lgbm_mod()

"""                 """

#print possible features
print('possible features:', X_train_scaled.columns.tolist(), '\n')

# Fit model, make predictions with all features
model.fit(X_train_scaled, Y_train)

preds_all = model.predict(X_test_scaled)

pdf = pred_df[['Status', 'FP']].copy()
pdf['Pred_FP_all'] = preds_all

# save the initial model to disk
filename = 'models/LGBM_models/nascar_model_allfeats.pkl'
joblib.dump(model, filename)

# get top 50 features
dset = pd.DataFrame({'attr':X_train_scaled.columns.tolist(),'importance':model.feature_importances_}).sort_values(by='importance', ascending=False).reset_index(drop=True)
attr50 = dset['attr'][0:50].tolist()

# Using Top 50 Features, Find Top 30 Features
model.fit(X_train_scaled[attr50], Y_train)
dset = pd.DataFrame({'attr':X_train_scaled[attr50].columns.tolist(),'importance':model.feature_importances_}).sort_values(by='importance', ascending=False).reset_index(drop=True)
attr30 = dset['attr'][0:30].tolist()

# Using Top 30 Features, Find Top 20 Features
model.fit(X_train_scaled[attr30], Y_train)
dset = pd.DataFrame({'attr':X_train_scaled[attr30].columns.tolist(),'importance':model.feature_importances_}).sort_values(by='importance', ascending=False).reset_index(drop=True)
attr20 = dset['attr'][0:20].tolist()

#Perform RFE (recursive feature elimination) using Top 20 Features, To Find Top 15
rfe_model = RFE(model, n_features_to_select = 15)
rfe_model.fit(X_train_scaled[attr20], Y_train)
dset = pd.DataFrame({'attr':X_train_scaled[attr20].columns.tolist(),'importance':rfe_model.ranking_}).sort_values(by='importance', ascending=False).reset_index(drop=True)
cols15 = dset[dset['importance']==1]['attr'].tolist()

#Perform RFE using Top 15 Features, To Find Top 10
rfe_model = RFE(model, n_features_to_select = 10)
rfe_model.fit(X_train_scaled[cols15], Y_train)
dset = pd.DataFrame({'attr':X_train_scaled[cols15].columns.tolist(),'importance':rfe_model.ranking_}).sort_values(by='importance', ascending=False).reset_index(drop=True)
cols10 = dset[dset['importance']==1]['attr'].tolist()

print('T50 features', attr50, '\n')
print('T30 features', attr30, '\n')
print('T20 features', attr20, '\n')
print('T15 features',cols15, '\n')
print('T10 features',cols10, '\n')

model.fit(X_train_scaled[attr50], Y_train)
preds50 = model.predict(X_test_scaled[attr50])
filename = 'models/LGBM_models/nascar_model_50feats.pkl'
joblib.dump(model, filename)

model.fit(X_train_scaled[attr30], Y_train)
preds30 = model.predict(X_test_scaled[attr30])
filename = 'models/LGBM_models/nascar_model_30feats.pkl'
joblib.dump(model, filename)

model.fit(X_train_scaled[attr20], Y_train)
preds20 = model.predict(X_test_scaled[attr20])
filename = 'models/LGBM_models/nascar_model_20feats.pkl'
joblib.dump(model, filename)

model.fit(X_train_scaled[cols15], Y_train)
preds15 = model.predict(X_test_scaled[cols15])
filename = 'models/LGBM_models/nascar_model_15feats.pkl'
joblib.dump(model, filename)

model.fit(X_train_scaled[cols10], Y_train)
preds10 = model.predict(X_test_scaled[cols10])
filename = 'models/LGBM_models/nascar_model_10feats.pkl'
joblib.dump(model, filename)


pdf['Pred_FP_50'] = preds50
pdf['Pred_FP_30'] = preds30
pdf['Pred_FP_20'] = preds20
pdf['Pred_FP_15'] = preds15
pdf['Pred_FP_10'] = preds10

pdf.to_csv('models/predictions_LGBM_50_30_20_15_10.csv')

feature_sets = ['all', '50', '30', '20', '15', '10']

mae_values = [
    "{:.2f}".format(mean_absolute_error(Y_test, preds_all)),
    "{:.2f}".format(mean_absolute_error(Y_test, preds50)),
    "{:.2f}".format(mean_absolute_error(Y_test, preds30)),
    "{:.2f}".format(mean_absolute_error(Y_test, preds20)),
    "{:.2f}".format(mean_absolute_error(Y_test, preds15)),
    "{:.2f}".format(mean_absolute_error(Y_test, preds10))
]

results_df = pd.DataFrame({'Features' : feature_sets, 'MAE' : mae_values})

results_df.style.hide_index()
results_df.to_clipboard()
display(results_df)

possible features: ['LL_yr_cumsum', 'LC_yr_cumsum', 'DNF_yr_cumsum', 'Win_yr_cumsum', 'T5_yr_cumsum', 'T10_yr_cumsum', 'T20_yr_cumsum', 'FinPos_mean_L3', 'StrtPos_mean_L3', 'PlaceDiff_mean_L3', 'LL_sum_L3', 'LL_mean_L3', 'LC_sum_L3', 'LC_mean_L3', 'Pts_sum_L3', 'Pts_mean_L3', 'PPts_sum_L3', 'PPts_mean_L3', 'FP_sum_L3', 'FP_mean_L3', 'DNF_sum_L3', 'DNF_sum_L10', 'Win_sum_L3', 'T5_sum_L3', 'T10_sum_L3', 'T20_sum_L3', 'Win_sum_L10', 'T5_sum_L10', 'T10_sum_L10', 'T20_sum_L10', 'GFP_sum_L3', 'GFP_mean_L3', 'GFTP_sum_L3', 'GFTP_mean_L3', 'QP_sum_L3', 'QP_mean_L3', 'Pct_QP_mean_L3', 'T15Laps_sum_L3', 'T15Laps_mean_L3', 'Pct_T15Laps_mean_L3', 'DRating_sum_L3', 'DRating_mean_L3', 'FinPos_mean_L3_track', 'StrtPos_mean_L3_track', 'PlaceDiff_mean_L3_track', 'LL_sum_L3_track', 'LL_mean_L3_track', 'LC_sum_L3_track', 'LC_mean_L3_track', 'Pts_sum_L3_track', 'Pts_mean_L3_track', 'PPts_sum_L3_track', 'PPts_mean_L3_track', 'FP_sum_L3_track', 'FP_mean_L3_track', 'DNF_sum_L3_track', 'Win_sum_L3_track', 'T5

,Features,MAE
0,all,12.57
1,50,12.72
2,30,13.40
3,20,13.78
4,15,14.47
5,10,15.37


In [60]:
running_pdf = pdf.loc[pdf['Status'] == 'running']
dnf_pdf = pdf.loc[pdf['Status'] != 'running']

In [61]:
running_pdf.describe()

,FP,Pred_FP_all,Pred_FP_50,Pred_FP_30,Pred_FP_20,Pred_FP_15,Pred_FP_10
count,1797.000000,1797.000000,1797.000000,1797.000000,1797.000000,1797.000000,1797.000000
mean,53.519421,50.121804,50.258395,49.682941,49.346005,49.065691,49.082641
std,18.939117,14.945478,14.887258,14.994239,14.880649,14.969273,14.030279
min,0.000000,1.597559,-1.212204,4.224986,2.844719,3.504189,3.718374
25%,39.700000,39.222676,39.594216,38.947845,39.212453,38.932570,39.578853
50%,53.400000,49.189698,49.223079,48.987875,48.779242,48.677088,48.425215
75%,66.000000,59.694684,60.571715,59.489339,59.101671,59.024376,57.866086
max,138.600000,99.895835,102.846859,101.802157,97.580567,107.513131,99.575307


In [62]:
dnf_pdf.describe()

,FP,Pred_FP_all,Pred_FP_50,Pred_FP_30,Pred_FP_20,Pred_FP_15,Pred_FP_10
count,281.00000,281.000000,281.000000,281.000000,281.000000,281.000000,281.000000
mean,18.48363,37.022825,36.385597,39.755529,41.760355,42.515667,41.907841
std,14.62147,16.680407,16.626566,16.477206,15.704815,15.162996,14.509840
min,0.00000,3.246549,3.428443,1.474170,2.339164,-1.602519,-0.444104
25%,6.10000,25.169427,25.222304,28.748809,32.117624,33.419128,32.832242
50%,15.90000,35.776640,34.316242,38.138513,40.616244,43.085428,42.881394
75%,28.10000,48.277195,46.603067,51.331614,52.087954,51.523042,51.598370
max,73.60000,85.119155,76.681222,85.095803,88.296765,81.227530,79.953563


In [63]:
print("{:.2f}".format(mean_absolute_error(running_pdf['Pred_FP_50'], running_pdf['FP'])))

11.45


In [64]:
print("{:.2f}".format(mean_absolute_error(dnf_pdf['Pred_FP_50'], dnf_pdf['FP'])))

20.85
